In [1]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from projectnb import ProjectContext, ProjectUtil
pc = ProjectContext.ProjectContext(sc, '48e1679f-100b-456b-87ab-0432209e9096', 'p-bed11d2793c7698686080191847c4f0cd0170229')


ImportError: No module named projectnb

In [1]:
sc._conf.set('spark.executor.memory', '16g')
sc._conf.set('spark.cores.max', '8')
sc._conf.set('spark.driver.allowMultipleContexts', 'true')
sc._conf.getAll()

#write to CSV
#data.repartition(1).write.format("com.databricks.spark.csv").option("header","true").mode("overwrite").save('/user-home/1014/branch_opp_data/binned_exportUS.csv')


[(u'spark.eventLog.enabled', u'true'),
 (u'spark.driver.extraJavaOptions',
  u'-Djavax.net.ssl.trustStore=/user-home/_global_/security/customer-truststores/cacerts'),
 (u'spark.app.name', u'my-notebook'),
 (u'spark.driver.host', u'172.16.112.61'),
 (u'spark.shuffle.service.enabled', u'true'),
 (u'spark.dynamicAllocation.executorIdleTimeout', u'300'),
 (u'spark.driver.port', u'39884'),
 (u'spark.driver.allowMultipleContexts', u'true'),
 (u'spark.executor.extraJavaOptions',
  u'-Djavax.net.ssl.trustStore=/user-home/_global_/security/customer-truststores/cacerts'),
 (u'spark.executor.id', u'driver'),
 (u'spark.eventLog.dir', u'/tmp/spark-events'),
 (u'spark.executor.memory', u'16g'),
 (u'spark.app.id', u'app-20180318013254-0361'),
 (u'spark.rdd.compress', u'True'),
 (u'spark.ui.enabled', u'false'),
 (u'spark.sql.warehouse.dir', u'/tmp'),
 (u'spark.serializer.objectStreamReset', u'100'),
 (u'spark.cores.max', u'8'),
 (u'spark.dynamicAllocation.initialExecutors', u'1'),
 (u'spark.master', u

In [4]:

import pandas as pd
import numpy as np
import pyspark as spark
from pyspark.sql import SparkSession, functions as F, Row, Column
from pyspark.sql.functions import datediff, unix_timestamp, from_unixtime

from pyspark.sql.types import DateType, StringType, StructField, IntegerType, FloatType, StructType
from datetime import date
from pyspark.sql.window import Window

from pyspark.sql.functions import udf
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

#https://stackoverflow.com/questions/36584812/pyspark-row-wise-function-composition
#https://stackoverflow.com/questions/40389433/pyspark-dataframe-apply-function-to-two-columns

<h3>Load DataFrames from DSX</h3>

In [4]:
# Access 'sq_lmi.txt' data file from the project.
sq_lmi = ProjectUtil.load_dataframe_from_file(pc, "sq_lmi.txt")
sq_lmi.show()
# Access 'Branch_Urbanicity.csv' data file from the project.
urban= ProjectUtil.load_dataframe_from_file(pc, "Branch_Urbanicity.csv")
urban.show()

+----+-------+----+
|  ID|LMI_HMT|SQFT|
+----+-------+----+
| 152|   null|4459|
| 145|   null|3000|
| 141|   null|3240|
| 143|   null|3400|
|2917|    LMI| 540|
| 117|    LMI|3000|
| 105|   null|3900|
| 142|   null|3000|
| 683|    LMI|2815|
| 725|   null|3277|
| 168|    LMI|1155|
| 208|   null|2500|
| 157|   null|6085|
| 209|   null|5832|
| 721| LMIHMT|3211|
| 549|   null|4155|
| 719| LMIHMT|4371|
|2924|   null| 555|
|6390|   null|4200|
|1747|   null|4200|
+----+-------+----+
only showing top 20 rows

+-----+--------------+--------------------+
|   ID|UrbanicityCode|      UrbanicityName|
+-----+--------------+--------------------+
| 3000|             6|               Rural|
|43328|             2|     Urban Periphery|
|  347|             4|  Suburban Periphery|
| 8873|             6|               Rural|
| 2900|             2|     Urban Periphery|
| 2884|             1|Principal Urban C...|
| 1260|             4|  Suburban Periphery|
| 2888|             5|          Semirurals|
|  343|   

<h3>Selecting, Viewing, Filtering, Aggregating</h3>

In [5]:
sq_lmi.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- LMI_HMT: string (nullable = true)
 |-- SQFT: integer (nullable = true)



In [6]:
sq_lmi.select('ID','SQFT').show(5)

+----+----+
|  ID|SQFT|
+----+----+
| 152|4459|
| 145|3000|
| 141|3240|
| 143|3400|
|2917| 540|
+----+----+
only showing top 5 rows



In [10]:
sq_lmi.select('ID','SQFT').filter(sq_lmi.ID <200).show(5)

+---+----+
| ID|SQFT|
+---+----+
|152|4459|
|145|3000|
|141|3240|
|143|3400|
|117|3000|
+---+----+
only showing top 5 rows



In [9]:
sq_lmi.filter(sq_lmi.ID <200).show(5)

+---+-------+----+
| ID|LMI_HMT|SQFT|
+---+-------+----+
|152|   null|4459|
|145|   null|3000|
|141|   null|3240|
|143|   null|3400|
|117|    LMI|3000|
+---+-------+----+
only showing top 5 rows



In [11]:
agged=sq_lmi.groupby('LMI_HMT').agg(F.min("SQFT").alias("min_sqft"),F.max("SQFT"),F.count('*'))
print(agged.show(5))


+-------+--------+---------+--------+
|LMI_HMT|min_sqft|max(SQFT)|count(1)|
+-------+--------+---------+--------+
|   null|     302|    37536|     797|
|    HMT|     522|     6000|      54|
|    LMI|     288|    16000|     186|
| LMIHMT|     598|    24930|     120|
+-------+--------+---------+--------+

None


In [12]:
#sql

sq_lmi.createOrReplaceTempView("tbl_1")
urban.createOrReplaceTempView("tbl_2")
spark = SparkSession.builder.getOrCreate()
sqlDf = spark.sql("select a.*, 'TEST' as dummy, b.UrbanicityCode from tbl_1 a inner join tbl_2 b on a.ID=b.ID where a.SQFT <5000")
print(type(sqlDf))
print(sqlDf.count())
sqlDf.show(5)

<class 'pyspark.sql.dataframe.DataFrame'>
960
+----+-------+----+-----+--------------+
|  ID|LMI_HMT|SQFT|dummy|UrbanicityCode|
+----+-------+----+-----+--------------+
| 152|   null|4459| TEST|             4|
| 145|   null|3000| TEST|             3|
| 141|   null|3240| TEST|             4|
| 143|   null|3400| TEST|             1|
|2917|    LMI| 540| TEST|             3|
+----+-------+----+-----+--------------+
only showing top 5 rows



In [13]:
#add new column (can chain together using where to filter as you go)
sq_lmi=sq_lmi.withColumn('SQFT_000', sq_lmi.SQFT/1000).withColumn('SQFT_ID', sq_lmi.SQFT/sq_lmi.ID)
print(sq_lmi.show(5))

sq_lmi=sq_lmi.drop('SQFT_000')
print(sq_lmi.show(5))

+----+-------+----+--------+-------------------+
|  ID|LMI_HMT|SQFT|SQFT_000|            SQFT_ID|
+----+-------+----+--------+-------------------+
| 152|   null|4459|   4.459| 29.335526315789473|
| 145|   null|3000|     3.0| 20.689655172413794|
| 141|   null|3240|    3.24|  22.97872340425532|
| 143|   null|3400|     3.4| 23.776223776223777|
|2917|    LMI| 540|    0.54|0.18512170037709977|
+----+-------+----+--------+-------------------+
only showing top 5 rows

None
+----+-------+----+-------------------+
|  ID|LMI_HMT|SQFT|            SQFT_ID|
+----+-------+----+-------------------+
| 152|   null|4459| 29.335526315789473|
| 145|   null|3000| 20.689655172413794|
| 141|   null|3240|  22.97872340425532|
| 143|   null|3400| 23.776223776223777|
|2917|    LMI| 540|0.18512170037709977|
+----+-------+----+-------------------+
only showing top 5 rows

None


<h3>Read in data FTP'd as a text file and create data frame</h3>

In [22]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

scale = (sqlContext.read.format("com.databricks.spark.csv").options(header= "true",delimiter=',').load("/user-home/1014/checking_models/scale.csv"))
print(type(scale))
scale.show(5)


AnalysisException: u'Path does not exist: file:/user-home/1014/checking_models/scale.csv;'

<h3>Read in with Pandas and transform to a DF</h3>

In [ ]:
pd.read_csv("/user-home/1019/scale.csv", sep=',')

<h3>Apply a function to a row of a data frame - adding an additional variable</h3>

In [14]:
from pyspark.sql.functions import udf, struct
from pyspark.sql.types import IntegerType,DoubleType

print (sq_lmi.printSchema())

#must check for missing values!



def calc_V1(row):
    if row is not None and row.SQFT_ID is not None and row.SQFT is not None :
        a=row.SQFT_ID*row.SQFT

        V1=a
        return V1
    else:
        return None




calc_V1_udf = udf(calc_V1, DoubleType())  #register the function and its return type

new_df = sq_lmi.withColumn("V1", calc_V1_udf(struct([sq_lmi[x] for x in sq_lmi.columns]))) #apply - the struct is needed to send the entire row
new_df.select('V1').show(5)

root
 |-- ID: integer (nullable = true)
 |-- LMI_HMT: string (nullable = true)
 |-- SQFT: integer (nullable = true)
 |-- SQFT_ID: double (nullable = true)

None
+------------------+
|                V1|
+------------------+
|130807.11184210525|
|62068.965517241384|
| 74451.06382978724|
| 80839.16083916085|
| 99.96571820363387|
+------------------+
only showing top 5 rows



<h3>Same as above but without UDF which is faster and you dont need to check for nulls</h3>

In [ ]:
new_df2=sq_lmi.withColumn("V1", sq_lmi.SQFT_ID*sq_lmi.SQFT ) 
new_df2.show(5)


<h3>Use this decorater technique from web site https://p058.github.io/p058.github.io/spark/2017/01/08/spark-dataframes.html</h3>

In [62]:
from pyspark.sql import SQLContext
from pyspark.sql.functions import udf
sqlContext = SQLContext(sc)

#dat = (sqlContext.read.format("com.databricks.spark.csv").options(header= "true",delimiter=',',nullValue="null").load("/user-home/1014/Jeff_data_pull_20180209.csv"))

dat = (sqlContext.read.csv(path="/user-home/1014/chicago_crime.csv",header=True,sep=',',inferSchema=True))
dat=dat.dropna()
print(type(dat))
dat.show(5)


<class 'pyspark.sql.dataframe.DataFrame'>
+-------+-----------+--------------------+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|     ID|Case Number|                Date|               Block|IUCR|Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+-------+-----------+--------------------+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|9955810|   HY144797|02/08/2015 11:43:...|   081XX S COLES AVE|1811|   NARCOTICS|POSS: CANNABIS 30...|       

In [74]:
skinny_dat=dat.select(['Community Area','X Coordinate','Y Coordinate'])
skinny_dat=skinny_dat.fillna(0)
print(skinny_dat.dtypes)
skinny_dat.show(5)

[('Community Area', 'int'), ('X Coordinate', 'int'), ('Y Coordinate', 'int')]
+--------------+------------+------------+
|Community Area|X Coordinate|Y Coordinate|
+--------------+------------+------------+
|            46|     1198273|     1851626|
|            53|     1178335|     1826581|
|            25|     1141717|     1898581|
|            25|     1137239|     1899372|
|            61|     1169986|     1873019|
+--------------+------------+------------+
only showing top 5 rows



In [60]:
def udf_wrapper(returntype):

    def udf_func(func):

        return udf(func, returnType=returntype)

    return udf_func


In [77]:
@udf_wrapper(StringType())
def community_over_40(x):

    return 'Yes' if x>40 else 'No'

In [79]:
skinny_dat = skinny_dat.withColumn("comm_40", community_over_40(skinny_dat['Community Area']))
skinny_dat.show(5)

+--------------+------------+------------+-------+
|Community Area|X Coordinate|Y Coordinate|comm_40|
+--------------+------------+------------+-------+
|            46|     1198273|     1851626|    Yes|
|            53|     1178335|     1826581|    Yes|
|            25|     1141717|     1898581|     No|
|            25|     1137239|     1899372|     No|
|            61|     1169986|     1873019|    Yes|
+--------------+------------+------------+-------+
only showing top 5 rows



<h3>Group by agg examples</h3>

In [89]:
#group by agg example

skinny_dat.groupby('comm_40').agg(F.min('X Coordinate')).show()


+-------+-----------------+
|comm_40|min(X Coordinate)|
+-------+-----------------+
|     No|          1119397|
|    Yes|          1100317|
+-------+-----------------+



In [173]:
#same, using UDF
#https://stackoverflow.com/questions/46187630/how-to-write-pyspark-udaf-on-multiple-columns

#quite complicated as no user defined agg function support in pyspark (the dev version does though)
#https://spark.apache.org/docs/1.1.1/api/python/pyspark.rdd.RDD-class.html#zipWithIndex

#so try with RDD
print(skinny_dat.rdd.take(10))


v=(skinny_dat
  .groupby("comm_40")
  .agg(F.collect_set("X Coordinate").alias("list_X")))

print(v.show())


z=v.rdd.map(lambda x: (x[0],min(x[1])))
print(z.toDF().show())

#or


def calc_min(data_list):
    
    min =data_list[0]
    
    for l in data_list:
        if l <min:
            min=l
    
    return min

calc_min_udf = udf(calc_min, IntegerType() )

z=v.withColumn('min_x', calc_min_udf('list_X'))
print(z.show())

#couldnt figure out how to pass rdd to a udf that was not lamnbda - maybe just tired :)




[Row(Community Area=46, X Coordinate=1198273, Y Coordinate=1851626, comm_40=u'Yes'), Row(Community Area=53, X Coordinate=1178335, Y Coordinate=1826581, comm_40=u'Yes'), Row(Community Area=25, X Coordinate=1141717, Y Coordinate=1898581, comm_40=u'No'), Row(Community Area=25, X Coordinate=1137239, Y Coordinate=1899372, comm_40=u'No'), Row(Community Area=61, X Coordinate=1169986, Y Coordinate=1873019, comm_40=u'Yes'), Row(Community Area=1, X Coordinate=1164732, Y Coordinate=1943222, comm_40=u'No'), Row(Community Area=19, X Coordinate=1135910, Y Coordinate=1914206, comm_40=u'No'), Row(Community Area=32, X Coordinate=1175384, Y Coordinate=1902088, comm_40=u'No'), Row(Community Area=32, X Coordinate=1175384, Y Coordinate=1902088, comm_40=u'No'), Row(Community Area=6, X Coordinate=1168809, Y Coordinate=1921459, comm_40=u'No')]
+-------+--------------------+
|comm_40|              list_X|
+-------+--------------------+
|     No|[1139776, 1177758...|
|    Yes|[1170585, 1189089...|
+-------+----

<h3>SQL join and case statement</h3>

In [3]:


user_schema = StructType([StructField("date", DateType(), True),
                               StructField("user", IntegerType(), True),
                         StructField("type", StringType(), True)])

In [4]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

tbl = (sqlContext.read.format("com.databricks.spark.csv").options(header= "true",delimiter=',',dateFormat="MM/dd/yyyy").schema(user_schema).load("/user-home/1014/test_sql_join.csv"))
print("Type: ",type(tbl))
print("Data Types: ",tbl.dtypes)
print('Number of rows: ',tbl.count())
tbl.show()

('Type: ', <class 'pyspark.sql.dataframe.DataFrame'>)
('Data Types: ', [('date', 'date'), ('user', 'int'), ('type', 'string')])
('Number of rows: ', 9)
+----------+----+------------+
|      date|user|        type|
+----------+----+------------+
|2017-01-12| 123|Email_change|
|2018-01-12| 123|Phone_change|
|2017-02-28| 123|Email_change|
|2017-02-13| 123|Email_change|
|2017-01-12| 456|Email_change|
|2017-01-12| 789|   NA_Change|
|2017-01-15| 789|   NA_Change|
|2017-01-12| 789|   NA_Change|
|2017-01-12| 123|Phone_change|
+----------+----+------------+



In [43]:
tbl.registerTempTable("tbl_1")
tbl.registerTempTable("tbl_2")


In [60]:

'''
join logic:

same user
the event to consider happens in same or a later month after the current row 
the event to consider happens at most 3 month after the current row 
the event to consider happens after the current row in terms of days 

'''



spark = SparkSession.builder.getOrCreate()

months_lookback = 3


query=            'select a.user, a.date, a.type, \
                  min(case when b.type = "Email_change" then datediff(b.date, a.date) end) as days_since_emails \
                  from tbl_1 a  left outer join tbl_2 b  \
                  on a.user=b.user  \
                  and months_between(b.date,a.date) >=0  \
                  and months_between(b.date,a.date) <="{}"  \
                  and b.date > a.date   \
                  group by a.user, a.date, a.type'.format(months_lookback)


sqlDf = spark.sql(query)

#sqlDf=sqlDf.fillna(months_lookback +1)                  
                  

In [61]:
print(sqlDf.count())
sqlDf.show()

8
+----+----------+------------+-----------------+
|user|      date|        type|days_since_emails|
+----+----------+------------+-----------------+
| 789|2017-01-12|   NA_Change|             null|
| 123|2018-01-12|Phone_change|             null|
| 123|2017-01-12|Phone_change|               32|
| 123|2017-01-12|Email_change|               32|
| 123|2017-02-13|Email_change|               15|
| 789|2017-01-15|   NA_Change|             null|
| 123|2017-02-28|Email_change|             null|
| 456|2017-01-12|Email_change|             null|
+----+----------+------------+-----------------+



<h3>Play with RDD</h3>


In [14]:
tbl.rdd.take(5)

[Row(date=datetime.date(2017, 1, 12), user=123, type=u'Email_change'),
 Row(date=datetime.date(2018, 1, 12), user=123, type=u'Phone_change'),
 Row(date=datetime.date(2017, 2, 28), user=123, type=u'Email_change'),
 Row(date=datetime.date(2017, 2, 13), user=123, type=u'Email_change'),
 Row(date=datetime.date(2017, 1, 12), user=456, type=u'Email_change')]

In [10]:
#col
tbl.rdd.map(lambda row: row[1]).take(5)

[123, 123, 123, 123, 456]

In [150]:
one=sc.parallelize(["a", "b", "c", "d"])
print(type(one))
print(one.take(10))
two=one.zipWithIndex()
print(two.take(10))


three=two.map(lambda x: (x[0],x[1],x[1]+1))
three.take(10)



<class 'pyspark.rdd.RDD'>
['a', 'b', 'c', 'd']
[('a', 0), ('b', 1), ('c', 2), ('d', 3)]


[('a', 0, 1), ('b', 1, 2), ('c', 2, 3), ('d', 3, 4)]

In [147]:
x = sc.parallelize([("a", ["apple", "banana", "lemon"]), ("b", ["grapes"])])
x.values().collect()
y=x.toDF()
print(y.dtypes)
y.show()

[('_1', 'string'), ('_2', 'array<string>')]
+---+--------------------+
| _1|                  _2|
+---+--------------------+
|  a|[apple, banana, l...|
|  b|            [grapes]|
+---+--------------------+



In [166]:
df = sqlContext.createDataFrame([
    ("a", 1, 2),
    ("a", 1, 4),
    ("b", None, 6),
], ["id", "code", "index"])
print(df.dtypes)
print(df.show())

v=(df
  .groupby("id")
  .agg(F.collect_set("code"), #will remove dups
       F.collect_list("index")))

print(v.show())


z=v.rdd.map(lambda x: (x[0],min(x[2])))
z.toDF().show()


[('id', 'string'), ('code', 'bigint'), ('index', 'bigint')]
+---+----+-----+
| id|code|index|
+---+----+-----+
|  a|   1|    2|
|  a|   1|    4|
|  b|null|    6|
+---+----+-----+

None
+---+-----------------+-------------------+
| id|collect_set(code)|collect_list(index)|
+---+-----------------+-------------------+
|  b|               []|                [6]|
|  a|              [1]|             [2, 4]|
+---+-----------------+-------------------+

None
+---+---+
| _1| _2|
+---+---+
|  b|  6|
|  a|  2|
+---+---+



<h3>windowing examples</h3>

In [5]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)



_schema = StructType([StructField("date", StringType(), True),
                               StructField("var1", IntegerType(), True),
                         StructField("var2", StringType(), True)])


test_list = [('2017-01-30',123,'A'),
('2017-01-17',123,'B'),
('2017-01-15',123,'A'),
('2017-01-15',123,'A'),
('2017-01-14',123,'A'),
('2017-01-11',123,'B'),
('2017-01-29',456,'A'),
('2017-01-22',789,'B'),
('2017-01-21',789,'B'),
('2017-01-20',789,'A'),
('2017-01-19',789,'A')

]

df = sqlContext.createDataFrame(test_list,schema=_schema) 
df=(df.withColumn('date',df.date.cast(DateType())))

#add this to have a numeric to use below 
df=(df.withColumn('dayssinceJan11900',datediff(df.date,F.lit(date(1900, 1, 1)))))
df.show(10)

+----------+----+----+-----------------+
|      date|var1|var2|dayssinceJan11900|
+----------+----+----+-----------------+
|2017-01-30| 123|   A|            42763|
|2017-01-17| 123|   B|            42750|
|2017-01-15| 123|   A|            42748|
|2017-01-15| 123|   A|            42748|
|2017-01-14| 123|   A|            42747|
|2017-01-11| 123|   B|            42744|
|2017-01-29| 456|   A|            42762|
|2017-01-22| 789|   B|            42755|
|2017-01-21| 789|   B|            42754|
|2017-01-20| 789|   A|            42753|
+----------+----+----+-----------------+
only showing top 10 rows



In [121]:
#count number of times each user had event in past 3 days

wSpec1=Window.partitionBy('var1').orderBy('dayssinceJan11900').rangeBetween(-3 ,-1)

df=(df.withColumn("events_past_3days", F.count(df.var2).over(wSpec1)))
df.show()




+----------+----+----+-----------------+-----------------+
|      date|var1|var2|dayssinceJan11900|events_past_3days|
+----------+----+----+-----------------+-----------------+
|2017-01-29| 456|   A|            42762|                0|
|2017-01-19| 789|   A|            42752|                0|
|2017-01-20| 789|   A|            42753|                1|
|2017-01-21| 789|   B|            42754|                2|
|2017-01-22| 789|   B|            42755|                3|
|2017-01-11| 123|   B|            42744|                0|
|2017-01-14| 123|   A|            42747|                1|
|2017-01-15| 123|   A|            42748|                1|
|2017-01-15| 123|   A|            42748|                1|
|2017-01-17| 123|   B|            42750|                3|
|2017-01-30| 123|   A|            42763|                0|
+----------+----+----+-----------------+-----------------+



In [122]:
#count number of times each user had event in past 3 days of type A
df=(df.withColumn("A_events_past_3days", F.sum((df.var2==('A')).cast(IntegerType())).over(wSpec1)))
df.show()

#null is returned when there are no records prior (in range) to a row
df=df.fillna(0,'A_events_past_3days')
df.show()

+----------+----+----+-----------------+-----------------+-------------------+
|      date|var1|var2|dayssinceJan11900|events_past_3days|A_events_past_3days|
+----------+----+----+-----------------+-----------------+-------------------+
|2017-01-29| 456|   A|            42762|                0|               null|
|2017-01-19| 789|   A|            42752|                0|               null|
|2017-01-20| 789|   A|            42753|                1|                  1|
|2017-01-21| 789|   B|            42754|                2|                  2|
|2017-01-22| 789|   B|            42755|                3|                  2|
|2017-01-11| 123|   B|            42744|                0|               null|
|2017-01-14| 123|   A|            42747|                1|                  0|
|2017-01-15| 123|   A|            42748|                1|                  1|
|2017-01-15| 123|   A|            42748|                1|                  1|
|2017-01-17| 123|   B|            42750|            

In [101]:
#count number of times each user had event in past 3 days when event is same as the current row
#????????#????????????????????????????????????????????????

df=(df.withColumn("same_events_past_3days", ).over(wSpec1)))
df.show()


In [6]:

wSpec1=Window.partitionBy('var1').orderBy('dayssinceJan11900').rangeBetween(-3 ,-1)

#keept the set to see what is there
df=(df.withColumn("events_past_3days_list", F.collect_list(df.var2).over(wSpec1)))

df.show()


+----------+----+----+-----------------+----------------------+
|      date|var1|var2|dayssinceJan11900|events_past_3days_list|
+----------+----+----+-----------------+----------------------+
|2017-01-29| 456|   A|            42762|                    []|
|2017-01-19| 789|   A|            42752|                    []|
|2017-01-20| 789|   A|            42753|                   [A]|
|2017-01-21| 789|   B|            42754|                [A, A]|
|2017-01-22| 789|   B|            42755|             [A, A, B]|
|2017-01-11| 123|   B|            42744|                    []|
|2017-01-14| 123|   A|            42747|                   [B]|
|2017-01-15| 123|   A|            42748|                   [A]|
|2017-01-15| 123|   A|            42748|                   [A]|
|2017-01-17| 123|   B|            42750|             [A, A, A]|
|2017-01-30| 123|   A|            42763|                    []|
+----------+----+----+-----------------+----------------------+

